# TP 4 de Statistique descriptive avec SAS

Objectifs de la séance : 
* Utiliser l'étape DATA afin de concaténer et fusionner des tables,
* Lire des tables dans un format différent de **sas7bdat** (en utilisant l'instruction INFILE, la procédure IMPORT ou les moteurs de l'instruction LIBNAME),
* Utiliser des formats,
* Réaliser une analyse univariée et une analyse bivariée.


>Le RMS Titanic est un paquebot transatlantique britannique qui fait naufrage dans l'océan Atlantique Nord en 1912 à la suite d'une collision avec un iceberg, lors de son voyage inaugural de Southampton à New York. Entre 1 490 et 1 520 personnes trouvent la mort, ce qui fait de cet événement l'une des plus grandes catastrophes maritimes survenues en temps de paix et la plus grande pour l'époque. (Source : Wikipédia)

Le but de ce TP est d'étudier les différences de caractériques entre les passagers et d'essayer de déterminer des profils particuliers parmi ceux qui ont survécu.

Pour se faire, nous disposons de quatre tables :

| Nom de la table | Extension |                           Descriptif                          |
|:---------------:|:---------:|:-------------------------------------------------------------:|
| titanic_client1 |    csv    | Informations relatives aux individus voyageant en 1ère classe |
| titanic_client2 |    csv    | Informations relatives aux individus voyageant en 2ème classe |
| titanic_client3 |    csv    | Informations relatives aux individus voyageant en 3ème classe |
|  titanic_place  |    xls    |            Informations sur la place des individus            |

Le dictionnaire des variables des tables *titanic_clientn* est :

|     Nom    |                   Libellé                   |
|:----------:|:-------------------------------------------:|
|     id     |                 Identifiant                 |
|  Civilite  |                   Civilité                  |
|     Age    |              Âge de l'individu              |
| EpouseFrat | Nombre de frères, soeurs ou époux.se à bord |
| ParentsEnf |     Nombre de parents ou enfants à bord     |


Le dictionnaire des variables de la table *titanic_place* est : 

|     Nom    |                              Libellé                              |
|:----------:|:-----------------------------------------------------------------:|
|     id     |                     Identifiant de l'individu                     |
|   Classe   |                   Classe du billet de l'individu                  |
|   Survie   | Indicatrice de survie (vaut 1 si l'individu a survécu et 0 sinon) |
|    Prix    |                    Prix du billet de l'individu                   |
| PortEmbarq |                 Port d'embarquement de l'individu                 |

Les données sont sur Moodle blabla

## Exercice 1 : Préparation des données

**Question 1** : Définissez deux bibliothèques *source* et *resultat*. Vous déposerez les quatre tables 

Est-ce que les quatre tables sont visibles dans la bibliothèque *source* ? Pourquoi ?

**Réponse** : Par défaut, le moteur utilisé dans l'instruction LIBNAME est le moteur V9 qui ne permet de lire que des tables crées dans une version postérieure à SAS 9.x.
Ici, il s'agit de fichiers au format *csv*.

In [ ]:
LIBNAME source "/folders/myfolders/Donnees_TP_SAS/TP4";
LIBNAME resultat "/folders/myfolders/Donnees_TP_SAS/TP4";

**Question 2** : À l'aide d'une étape DATA et des instructions INFILE et INPUT, créez les tables *donnees_c1* (à l'aide de la table titanic_client1.csv) et *donnees_c2* (à l'aide de la table titanic_client2.csv) dans la bibliothèque Work. On utilisera une étape DATA par table.

In [ ]:
DATA donnees_C1;
INFILE "/folders/myfolders/Donnees_TP_SAS/TP4/titanic_client1.csv" FIRSTOBS = 2 DLM = "," DSD MISSOVER;
INPUT id $ Civilite $ Age EpouseFrat ParentsEnf;
RUN;

In [ ]:
DATA donnees_C2;
INFILE "/folders/myfolders/Donnees_TP_SAS/TP4/titanic_client2.csv"  FIRSTOBS = 2 DLM = "," DSD MISSOVER;
INPUT id $ Civilite $ Age EpouseFrat ParentsEnf;
RUN;

**Question 3** : À l'aide d'une procédure IMPORT, créez la table donnees_c3 dans la bibliothèque Work. 

In [ ]:
PROC IMPORT DATAFILE = "/folders/myfolders/Donnees_TP_SAS/TP4/titanic_client3.csv" OUT = donnees_c3 DBMS = CSV;
DELIMITER = ",";
RUN;

**Question** : Sachant que l'identifiant est une variable texte de la forme "idxxxx", vérifiez en utilisant une procédure la taille des variables de la table *donnees_c3*.

In [ ]:
PROC CONTENTS DATA = donnees_c3;
RUN;

**Réponse** : L'identifiant *id* dans cette table est de longueur 5 alors qu'il est censé être de longueur 6 (car de la forme "idxxxx"). Il faut relancer une procédure IMPORT en utilisant GUESSINGROWS. De base, dans la procédure PROC IMPORT, SAS lit un certain nombre d'observations pour déterminer les longueurs des variables. Par exemple, si les deux premieres observations ont comme identifiant *id101* et *id102* alors SAS va assigner une longueur de 5 à cette variable même si la suivante est *id1000*.

En utilisant l'instruction GUESSINGROWS, on peut lui indiquer le nombre de lignes à lire pour déterminer la longueur des variables.

**Question** : Relancez la procédure IMPORT en utilisant l'instruction GUESSINGROWS.

In [ ]:
PROC IMPORT DATAFILE = "/folders/myfolders/Donnees_TP_SAS/TP4/titanic_client3.csv" OUT = donnees_c3 DBMS = CSV REPLACE;
/* il est nécessaire d'ajouter l'option de procédure REPLACE pour écraser l'ancienne table */
DELIMITER = ",";
GUESSINGROWS = 708; /* qui correspond au nombre d'observations de la table */
RUN;

**Question 4** : À l'aide d'une étape DATA, concaténez les tables donnees_c1, donnees_c2 et donnees_c3 en une table donnees_client. Cette table sera stockée dans la bibliothèque *source*.

On pourra modifier la longueur de la variable *id* des tables donnees_c1 et donnees_c2.

In [ ]:
DATA source.donnees_client;
SET donnees_c1 donnees_c2 donnees_c3;
RUN;

**Question** : Créez la bibliothèque *donneesp* pointant vers le fichier *titanic_place.xlsx*. On utilisera le moteur XLSX.
Puis à l'aide d'une étape DATA, créez une copie de la table dans la bibliothèque *source* qui nommera donnees_place.

In [ ]:
LIBNAME donneesp XLSX "/folders/myfolders/Donnees_TP_SAS/TP4/titanic_place.xlsx";

DATA source.donnees_place;
SET donneesp.titanicplace;
RUN;

**Question** : La table donnees_client contient un doublon. Supprimez-le en utilisant une procédure SORT et l'option de procédure NODUPKEY.

In [ ]:
PROC SORT DATA = source.donnees_client NODUPKEY;
BY id;
RUN;

**Réponse** : On lit dans la LOG que "NOTE: 1 observations with duplicate key values were deleted."

**Question** : À l'aide d'une étape DATA, fusionnez les tables donnees_client et donnees_place. On utilisera la variable *id* comme clef et on ne gardera que les observations dont l'identifiant est présent dans les deux tables (en cours de BDR, on parle d'*inner join*). La table sera stockée dans la bibliothèque *source* et se nommera *titanic*.

In [ ]:
DATA source.titanic;
MERGE source.donnees_client(in=client) source.donnees_place(in=place);
IF client and place THEN OUTPUT;
RUN;

**Question** : À l'aide d'une procédure MEANS, affichez le nombre de valeurs manquantes pour la variable Age.

Créez une table titanic_svm dans la bibliothèque *resultat* qui contiendra uniquement les observations de la table *titanic* de la bibliothèque *source* dont l'âge n'est pas manquant.

In [ ]:
PROC MEANS DATA = source.titanic NMISS;
VAR AGE;
RUN;

In [ ]:
DATA resultat.titanic_svm;
SET source.titanic;
IF NOT(MISSING(AGE)) THEN OUTPUT;
RUN;

**Question** : À l'aide d'une étape DATA, crééz une table titanic_appur dans la bibliothèque *resultat* qui contiendra uniquement les observations de la table *titanic* de la bibliothèque *source* dont toutes les variables

In [ ]:
DATA resultat.titanic_appur;
SET resultat.titanic_svm;
IF NOT(MISSING(AGE) | MISSING(CIVILITE) | MISSING(CLASSE) | MISSING(EPOUSEFRAT) | MISSING(PARENTSENF) | MISSING(PRIX) | MISSING(SURVIE)) THEN OUTPUT;
RUN;

**Question** : Créez un format *$sexe* qui associera pour chaque titre de civilité, le sexe de l'individu ("Homme" ou "Femme").

On fera les hypothèses suivantes : 

| Civilite |    Libellé   |        Hypothèse        |
|:--------:|:------------:|:-----------------------:|
|   ColM   |   Colonnel   |  Il s'agit que d'hommes |
|    DrF   |   Docteure   | Il s'agit que de femmes |
|    DrM   |    Docteur   |  Il s'agit que d'hommes |
|  Master  |    Maître    |  Il s'agit que d'hommes |
|   Miss   | Mademoiselle | Il s'agit que de femmes |
|    Mr    |   Monsieur   |  Il s'agit que d'hommes |
|    Mrs   |    Madame    | Il s'agit que de femmes |
|    Ms    |    Madame    | Il s'agit que de femmes |

In [ ]:
PROC FORMAT;
    VALUE $sexe
        "ColM" = "Homme"
        "DrF" = "Femme"
        "DrM" = "Homme"
        "Master" = "Homme"
        "Miss" = "Femme"
        "Mr" = "Homme"
        "Mrs" = "Femme"
        "Ms" = "Femme";
RUN;

**Question** : Ajoutez une variable sexe à la table titanic_appur de la bibliothèque *resultat* qui contiendra le sexe de chaque individu. On pourra utiliser le format *$sexe*.

In [ ]:
DATA resultat.titanic_appur;
SET resultat.titanic_appur;
SEXE = PUT(Civilite, $sexe.);
RUN;

## Exercice 2 : Analyse de la table *titanic_appur*

**Question** : Combien d'individus ont survécu ? Quelle part de la population totale représentent ils ?

In [ ]:
PROC FREQ DATA = source.titanic_appur;
TABLE Survie;
RUN;

**Question** : Quel est l'âge moyen des individus ? Donnez l'âge moyen par sexe également.

In [ ]:
PROC MEANS DATA = source.titanic_appur MEAN;
VAR AGE;
CLASS SEXE;
RUN;

**Question** : Donnez les quartiles de la distribution des prix des tickets ainsi que le minimum et le maximum. Rappelez la définition de la VAI et de la VAS. Donnez les valeurs extrêmes.

In [ ]:
PROC MEANS DATA = source.titanic_appur Q1 MEDIAN Q3 MIN MAX;
VAR PRIX;
RUN;

# Attention la variable PRIX a été codée en char et pas un numérique ...